# zhsegment: default program

In [1]:
from default import *

## Run the default solution on dev

In [5]:
Pw = Pdist(data=datafile("data/count_1w.txt"))
segmenter = Segment(Pw)
output_full = []
with open("data/input/dev.txt",'r', encoding='UTF-8') as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:5]))

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。
上午 在 这里 签字 的 是 知识 信息网 络通讯技 术和脱氧 核 糖 核 酸 生物 技术 两 个 项目 ， 同时 还 签订 了 语言 教学 交流 合作 协议 。
这 三 个 项目 是 分别 由 国务院 发展 研究 中心 国际 技术 经济 研究所 上海 分 所 和 上海市 浦东 继续 教育 中心 ， 与 美国 知识 信息 网络 公司 、 世界 学习 组织 、 海 赛克 公司 签订 的 。


## Evaluate the default output

In [3]:
from zhsegment_check import fscore
from zhsegment_check import precision
from zhsegment_check import recall
with open('data/reference/dev.out','r', encoding='UTF-8') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    pn = precision(ref_data, output_full)
    rl = recall(ref_data, output_full)
    print("precision: {:.2f}".format(pn), file=sys.stderr)
    print("recall: {:.2f}".format(rl), file=sys.stderr)
    print("score: {:.2f}".format(tally), file=sys.stderr)


precision: 0.89
recall: 0.92
score: 0.91


## Documentation

Write some beautiful documentation of your program here.

# inicialize the Heap

Note: below are only for demonstrating, not for running. Please ignore the error report.

In [ ]:
heap=[]
heap = self.MatchWords(0, text)
heap.sort(key=lambda a: a[2])

# the match word module
In the matched input module, we use the start position of the input text, to add new words to the word candidate pool. But only those word that is occurred in the 1w text will be selected out. 

In [ ]:
def MatchWords(self, startPosition, text):
    entryList = []
    L = min((self.Pw).maxLength, len(text) - startPosition)  # L = min(maxlen, j) in order to avoid long words
    for i in range(L):
        currword = text[startPosition : startPosition + i + 1] # the current text segmentation
        if currword in self.Pw:
            entryList.append((currword, startPosition, log10(self.Pw(currword)), None)) # add the current entry

    # if words not in dictionary
    if len(entryList) == 0:
        for i in range(len(text) - startPosition):
            if(i < 4):
                currword = text[startPosition : startPosition + i + 1]  
                entryList.append((currword, startPosition, log10(self.Pw(currword)), None)) 
    return entryList

# Implement the pseudocode part.

The following is an example of how we construct the chart. For example, our algorithm told us "签订" has higher probability then “签", so it is popped out first to fill in the index 5 position. Then the “签" is popped out to compare with the position of "签订", and did not win back the position, so the position 5 is still "签订"'s, which will be back-pointed to by the next word "高" in the chart.

final chart[ 0 ]: chartEntry (start=中, end=1 logprob=-2.448512, backptr=None)

final chart[ 1 ]: chartEntry (start=美, end=2 logprob=-5.787403, backptr=0)

final chart[ 2 ]: chartEntry (start=在, end=3 logprob=-7.741467, backptr=1)

final chart[ 3 ]: chartEntry (start=沪, end=4 logprob=-12.069362, backptr=2)

final chart[ 4 ]: chartEntry (start=签, end=5 logprob=-16.397257, backptr=3)

final chart[ 5 ]: chartEntry (start=签订, end=6 logprob=-15.744045, backptr=3)

final chart[ 6 ]: chartEntry (start=高, end=7 logprob=-18.594818, backptr=5)

final chart[ 7 ]: chartEntry (start=订高科, end=8 logprob=-33.327212, backptr=4)

final chart[ 8 ]: chartEntry (start=科技, end=9 logprob=-21.570531, backptr=6)

final chart[ 9 ]: chartEntry (start=合, end=10 logprob=-26.023365, backptr=8)

final chart[ 10 ]: chartEntry (start=合作, end=11 logprob=-24.206020, backptr=8)

final chart[ 12 ]: chartEntry (start=协议, end=13 logprob=-27.316431, backptr=10)

中 美 在 沪 签订 高 科技 合作 协议


In [ ]:
while (len(heap) != 0):

    # pop the largest prob entry from heap
    entry = heap[-1]
    heap = heap[:-1]

    # find endindex of current text segmentation / startindex of next text segmentation
    endindex = entry[1] + len(entry[0]) - 1

    # check chart[endindex] has preventry or not
    if chart[endindex] is not None:
        prevEntry = chart[endindex]
        if entry[2] > prevEntry[2]:    # check the prob of two entries
            chart[endindex] = entry
        else:
            continue
    else:
        chart[endindex] = entry

        entryList = self.MatchWords(endindex + 1, text)
        for currEntry in entryList:
            newEntry = (currEntry[0], currEntry[1], currEntry[2] + entry[2], endindex) # update prob 
            if newEntry not in heap:
                heap.append(newEntry)
        heap.sort(key=lambda a: a[2])


# Found the chart and output the segmentation result

In [ ]:
# use back pointer to trace the segmentation
finalEntry = chart[-1]
segmentation = []
segmentation.append(finalEntry[0])
while finalEntry[3] is not None:
    finalEntry = chart[finalEntry[3]]
    segmentation.append(finalEntry[0])
segmentation.reverse()

return segmentation


# extra function
we add this line "self.maxLength = max(map(len, self.keys()))" to count the max length of the word in 1w txt. Also, we add the function in assignment 0 which makes the unseen words a very small probability.

In [ ]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."

    def __init__(self, data=[], N=None, missingfn=None):
        for key, count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.maxLength = max(map(len, self.keys()))
        self.missingfn = missingfn or (lambda k, N: 1./(N*8000**len(k))) # #same reason as A0, avoid long words


# some equations related to code

argmax the production of likelyhood

In [ ]:
def product(nums):
    "Return the product of a sequence of numbers."
    return reduce(operator.mul, nums, 1)

segmentation iteration

In [ ]:
# check chart[endindex] has preventry or not
if chart[endindex] is not None:
    prevEntry = chart[endindex]
    if entry[2] > prevEntry[2]:    # check the prob of two entries
        chart[endindex] = entry
    else:
        continue
else:
    chart[endindex] = entry

calculate the maximun length of the word in 1w txt, and choose the smaller one between it and the remaining length of the text we are processing.

In [ ]:
L = min((self.Pw).maxLength, len(text) - startPosition)  # L = min(maxlen, j) in order to avoid long words

assign small probability to unknown word in 1w txt

In [ ]:
self.missingfn = missingfn or (lambda k, N: 1./(N*8000**len(k))) # #same reason as A0, avoid long words

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

in this example result, our method did not perform well:
    "上午 在 这里 签字 的 是 知识 信息网 络通讯技 术和脱氧 核 糖 核 酸 生物 技术 两 个 项目 ， 同时 还 签订 了 语言 教学 交流 合作 协议 。"
    
A better method could be to make use of the 2w txt with the bigram consideration.

We tried to use the smoothing method for the unseen words in the 1w txt, but it does not work in our code, with F1 score 0.06.